---
toc: true
comments: false
layout: post
title: Individual Review - Advik G
description: Recap of key events and learning
type: tangibles
courses: { compsci: {week: 10} }
---

# Issue Recap
I used [GitHub Issues](https://github.com/APCSP-RAGS/awsRAGS_flask/issues) to organize work for me and my partner, Srijan:
- [Starting Checklist](https://github.com/APCSP-RAGS/awsRAGS_flask/issues/2): This helped us stay on task and get organized while in the starting stages of development
- [Commit Summaries](https://github.com/APCSP-RAGS/awsRAGS_flask/issues/3): This summary was updated for each relevant commit. This is incredibly useful for charting our progress and looking at each commit easily. THe supplied text also served as more detailed commit messages. In general, this issue was used the most for organizing work done each day.

We didn't have many other issues as most of our task delegation was in communication through Slack and/or the checklist issue.

Grading Issues:
- [Self Grade Issue](https://github.com/APCSP-RAGS/pp-frontend/issues/6) - Issue used for personal grading
- [Eshika's Group - Lung Cancer](https://github.com/EshikaP1/frontend/issues/1#issuecomment-1791879996) - Peer grade from dress rehearsal
- [Lincoln's Group - Frogs](https://github.com/IshanCornick/FrontendRepo/issues/1#issuecomment-1792745899) - Peer grade from dress rehearsal
- [Nitin's Group - Colleges](https://github.com/CollegeRankings/CollegeRankingsBackend/issues/44#issuecomment-1791883099) - from N@TM
- [Will's Group - Geoguessr](https://github.com/rliao569/Frontend-CSP/issues/15#issuecomment-1791887511) - from N@TM
- [Hayden's Group - Ciphers](https://github.com/monke7769/passion/issues/6#issuecomment-1791892408) - from N@TM

# Backend Deployment
I had a lot of struggles deploying our backend to AWS in the beginning, so I made a [blog post](https://advikg.github.io/student/2023/10/08/AWS-Struggles_IPYNB_2_.html) a while ago to explain what I faced, and hopefully help other people with the same issues.


## DNS
Registering a domain should've been a relatively easy process, but there were many struggles that people had, including me. First off, the AWS Route 53 layout looks complicated, to say the least. But when you find the correct spot by going to Hosted Zones > stu.nighthawkcodingsociety.com > Create Record, there's some problems in the directions. It tells you to create a CNAME record pointing to the domain stu.nighthawkcodingsociety.com, but what this does is redirect first to the site domain, then to the site IP. This chain redirection messes with some web interactions, so it's better to use the static IP address of the server: `3.130.255.192`
- Also, make sure **not** to edit anything else, it's best not to change default settings unless you know exactly what you're doing.

One more thing:
A **lot** of people have been using a third-party website called `Duck DNS` for their domains. While this isn't necessarily a bad thing, using Route 53 isn't that complicated, will allow classmates and Mr. Mortenson/Mr. Lopez to help troubleshoot your site, and helps them manage your sites easily. I'd highly suggest using Route 53 over Duck DNS.

## AWS/Docker Deployment
There's a few things that people needed to keep in mind when finally deploying their backend:
1. Make sure your port is **defined** in the docker files and **available** on the AWS instance(use `docker ps` to view ports currently in use)
2. Make sure that running `docker-compose up` on your host in the correct directory runs the webserver with no issue, on the right port
3. Once you are **finished** with the first few steps, you can clone the repo on the AWS instance and run the docker build
4. Make sure to tailor the NGINX configurations specifically towards your site, and don't forget to test the configs using `nginx -t`
5. After reloading the configs, make sure your domain is listed in the output for `certbot --nginx`
A problem I had was that my domain was not listed in the certbot output. I used ChatGPT for help and found that you can use certbot for a specific domain by running `sudo certbot --nginx -d ----.stu.nighthawkcodingsociety.com`(insert the right domain name there).
I found out that since my domain name had an underscore(_), certbot looked at it as invalid. I was able to quickly solve this by changing the domain in Route 53 and in the NGINX configurations, and reloading everything.

## General Tips
- Use sudo for all of your commands, this gets rid of any permission errors you might have
- Read the errors you have, don't just bug someone else to figure it out for you. ChatGPT and google are great tools for troubleshooting errors, and this way you can also learn more
- Make sure to reload nginx configurations or redo the docker build if you change any configurations
- Try not to mess with the instance(Putting random stuff in .bashrc, using a forkbomb, making annoying cronjobs), it makes it a lot less fun for everybody

Well, that's all I have. You can view our groups backend site [here](https://awsrags-flask.stu.nighthawkcodingsociety.com)

# Backend Development
Alright, now onto the actual development of the backend. First going into this, I was very confused. I slowly started to understand the structure of the server after experimenting with the Jokes API to make a `Create` method.

## Testing with Jokes
I first started with the `jokes.py` file in the `model` folder, where I defined a function that added a joke to the dictionary:

In [ ]:
# Function to add jokes(for create method)
def createJoke(joke):
    item_id = len(jokes_data)
    jokes_data.append({"id": item_id, "joke": joke, "haha": 0, "boohoo": 0})

Then, I added some code to use the JSON request body in the API endpoint:

In [ ]:
class JokesAPI:
    # Method to create joke, 
    class _Create(Resource):
        def post(self): # Accept POST requests
            data = request.get_json() # Get JSON data from the request body
            print(data)
            joke = data.get('joke')
            if joke is not None: # Error handling
                createJoke(joke) # Actually running the method
                return jsonify(jokes_data[-1])
            else:
                return jsonify({'error': 'Invalid data'})
    api.add_resource(_Create, '/create') # Add the method to the /create endpoint

I was able to test this using Postman:
![Postman POST Test]({{site.baseurl}}/images/postman_jokescreate.png)

## Character Songs API

For our project, we had two main themes we wanted to incorporate: Breaking Bad, and Music
Now they seem like completely unrelated things, but I was able to tie them together by making this API, which:
- Uses SQLAlchemy to persisitently store data
- Has a list of songs with artist and genre, for characters from Breaking Bad
- Has both Create and Read methods in order to increase functionality
- Gets lyrics using the `lyricgenius` API for each song in the table

In our model, we defined a class with all of our data points:

In [ ]:
class Song(db.Model): # Create class
    __tablename__ = "Song"
    id = db.Column(db.Integer, primary_key=True)  # Define a primary key column
    character = db.Column(db.String, nullable=False) # Breaking bad character
    song_name = db.Column(db.String, nullable=False)
    artist = db.Column(db.String, nullable=False)
    genre = db.Column(db.String, nullable=False)
    lyrics = db.Column(db.String, nullable=False)
    def __init__(self, character, song_name, artist, genre, lyrics): # Constructer 
        self.character = character
        self.song_name = song_name
        self.artist = artist
        self.genre = genre
        self.lyrics = lyrics
    # Convert db data to a dictionary in order to return easily using JSON, used for read method
    def to_dict(self):
        return {"character": self.character, "song_name": self.song_name, "artist": self.artist, "genre": self.genre, "lyrics": self.lyrics}
    # Create method to let users add a song to the DB
    def create(self):
        try:
            db.session.add(self)  # add prepares to persist object to table
            db.session.commit()  # SQLAlchemy requires a manual commit
            return self
        except: 
            db.session.remove() # remove object from table if invalid
            return None
        # Read method to return every part of the table
    def read(self):
        return {
            "id": self.id,
            "character": self.character,
            "song_name": self.song_name,
            "artist": self.artist,
            "genre": self.genre,
            "lyrics": self.lyrics
        }

In this I created a table with different columns to sort each data type, then defined methods to both read from and add to the database. These methods are very useful when trying to integrate CRUD in the frontend. But now that the table is created, it needs data. This is where the `initSongs()` function comes into the spotlight. This function is called at the start of running the server, and initializes the database with data. For some projects, this data can be webscraped or gotten through a 3rd party API, but for this we just inputted manual data since there wasnt any clear way to automate it. We also depended on user input from the create method in order to populate the data more, and this worked pretty well since the data is stored persistently in the SQLite DB.

In [ ]:
class SongAPI:
    class _Create(Resource):
        def post(self):
            # get JSON request body
            body = request.get_json()
            # parse JSON to get variables
            character = body.get('character')
            song_name = body.get('song_name')
            artist = body.get('artist')
            genre = body.get('genre')
            lyrics = body.get('lyrics')
            # Set up Song class object
            song_obj = Song(character=character, song_name=song_name, artist=artist, genre=genre, lyrics=lyrics)
            
            ''' #2: Key Code block to add song to database '''
            # use method defined in model to create song in database
            song = song_obj.create()
            # success returns json of song
            if song:
                return jsonify(song.read()) # Use read method to show everything
            # failure returns error
            return {'message': f'Invalid input, correct fields should be character, song_name, lyrics, artist, and genre'}, 400

            
    class _Read(Resource):
        def get(self):
        # Retrieve all songs from the database
            songs = Song.query.all()
            json_ready = [song.to_dict() for song in songs]
        # Return the JSON response
            return jsonify(json_ready)
    # building RESTapi resources/interfaces, these routes are added to Web Server
    api.add_resource(_Create, '/create')
    api.add_resource(_Read, '/')


In the API, it was pretty simple to define a read and create method to let the frontend access and write to the DB. We used `jsonify` for ease of use and simple integration. The create method parses the JSON request body and uses the method defined in the model to add the song to the DB and generate lyrics. This API structure allowed us to format requests easily, as shown here:
![Song Create]({{site.baseurl}}/images/songcreate_postman.png)

# Team Teach
Past the passion project work, I also worked on developing the code used in our team teach for CB 3.7 & 3.8: Iteration. 